In [6]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [7]:
#数据处理
def dataset(data_type='train', batch_size=10):
    # 创建队列保护输入文件列表
    filename_queue = tf.train.string_input_producer(['data/data_tfrecord/' + data_type + '.tfrecords'])  
    # 读取并解析一个tfrecord
    reader = tf.TFRecordReader()  
    _, serialized_example = reader.read(filename_queue)
    example = tf.parse_single_example(serialized_example, features={
        'f1': tf.FixedLenFeature([], tf.string),
        'f2': tf.FixedLenFeature([], tf.string)})
    # 将字符串分别解析成标签和图像对应的像素组（并重组）
    image = tf.decode_raw(example['f1'], tf.float32)
    label = tf.decode_raw(example['f2'], tf.float64)
    image = tf.reshape(image, [28, 28])
    label= tf.reshape(label,[1,10])
    
    img_batch, l_batch = tf.train.shuffle_batch(
        [image, label],
        batch_size=batch_size,
        capacity= (3 * batch_size),
        min_after_dequeue=batch_size)
    return img_batch, l_batch

In [8]:
def change(Y):
    s,d,f = Y.shape
    b = np.empty((s,d*f))
    for i in range(s):
        c=Y[i]
        b[i]=c.reshape(d*f)
    Y=b
    return Y

In [9]:
def KNN(test_vec, train_data, train_label, k):  
    train_data_size = train_data.shape[0]
    #将test_vec在第0轴复制train_data_size倍，在第1轴复制1倍，并将复制后的与train_data相减
    dif_mat = np.tile(test_vec, (train_data_size,1)) - train_data
    sqr_dif_mat = dif_mat ** 2  
    #第1轴（横）每行向量相加（纵轴为第0轴）
    sqr_dis = sqr_dif_mat.sum(axis = 1) 
    distance = sqr_dis ** 0.5
    #返回数组值由小到大的索引值
    sorted_idx = distance.argsort()
    #确定距离最小的k个值都属于哪个类别，类别里个数最高的为分类结果
    class_cnt = {}  
    maxx = 0  
    best_class = 0  
    for i in range(k):  
        tmp_class = train_label[sorted_idx[i]].argmax(axis=0)
        tmp_cnt = class_cnt.get(tmp_class, 0) + 1  
        class_cnt[tmp_class] = tmp_cnt  
        if(tmp_cnt > maxx):  
            maxx = tmp_cnt  
            best_class = tmp_class  
    return best_class  

In [10]:
def run_KNN():
    train_img, train_label = dataset(batch_size=30000)
    test_img, test_label = dataset(batch_size=200)
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)  
        coord = tf.train.Coordinator() 
        threads=tf.train.start_queue_runners(sess=sess,coord=coord)
        train_img, train_label = sess.run([train_img, train_label])
        train_label = change(train_label)
        train_img = change(train_img)
        l = test_img.get_shape().as_list()
        tot =l[0]
        err = 0
        print("testing...")
        for i in range(tot):
            test_i, test_l = sess.run([test_img, test_label])
            test_i = change(test_i)
            test_l = change(test_l)
            best_class = KNN(test_i[i], train_img, train_label, 3)
            if(best_class != test_l[i].argmax(axis=0)):  
                err = err + 1
        print("ok")
        print("accuracy:") 
        print(1 - err / tot)
        coord.request_stop()
        coord.join(threads)
            
run_KNN()

testing...
ok
accuracy:
0.9
